### Hedging: Volatility Mismatch

Consider a short position in a European call option on a non-dividend paying stock with a maturity of one year and strike K = 99 EUR. Let the one-year risk-free interest rate be 6% and the current stock price be 100 EUR. Furthermore, assume that the volatility is 20%.

Use the Euler method to perform a hedging simulation.

#### 1. Matching Volatility

Conduct an experiment where the volatility in the stock price process matches the volatility used in the delta computation (i.e., both set to 20%). Vary the frequency of hedge adjustments (from daily to weekly) and explain the results.

#### 2. Mismatched Volatility

Perform numerical experiments where the volatility in the stock price process does not match the volatility used in the delta valuation. Run computational experiments for various levels of volatility and discuss the outcomes.